In [ ]:
from typing import Set, Tuple
from functools import cached_property

class IndependenceRelation:
    def __init__(self, left: Set, right: Set, given: Set):
        self.left = frozenset(left)   # Using frozenset for hashability
        self.right = frozenset(right)
        self.given = frozenset(given)

    def __eq__(self, other):
        return (self.left, self.right, self.given) == (other.left, other.right, other.given)

    def __hash__(self):
        return hash((self.left, self.right, self.given))

    def __repr__(self):
        left_str = ", ".join(self.left)
        right_str = ", ".join(self.right)
        given_str = ", ".join(self.given)
        return f"({left_str} ⊥ {right_str}) | {given_str}"


class SemigraphoidModel:
    def __init__(self, relations: Set[IndependenceRelation]):
        self.relations = relations

    @cached_property
    def minimized_relations(self) -> Set[IndependenceRelation]:
        # Initial reduction using symmetry
        reduced = self.apply_symmetry(self.relations)

        # Iterative application of decomposition, weak union, and contraction
        while True:
            new_reduced = self.apply_decomposition(reduced)
            new_reduced = self.apply_weak_union(new_reduced)
            new_reduced = self.apply_contraction(new_reduced)
            
            if new_reduced == reduced:
                break
            reduced = new_reduced

        return reduced

    def apply_symmetry(self, relations: Set[IndependenceRelation]) -> Set[IndependenceRelation]:
        new_relations = set(relations)
        for relation in relations:
            new_relation = IndependenceRelation(relation.right, relation.left, relation.given)
            new_relations.add(new_relation)
        return new_relations


    def apply_decomposition(self, relations: Set[IndependenceRelation]) -> Set[IndependenceRelation]:
        new_relations = set(relations)
        for relation in relations:
            left = relation.left
            right = relation.right
            given = relation.given
            for var in given:
                new_relation = IndependenceRelation(left | {var}, right, given - {var})
                new_relations.add(new_relation)
        return new_relations


    def apply_weak_union(self, relations: Set[IndependenceRelation]) -> Set[IndependenceRelation]:
        new_relations = set(relations)
        for relation1 in relations:
            if relation1.right != relation1.given:
                continue
            for relation2 in relations:
                if relation2.right != relation2.given:
                    continue
                if relation1.left == relation2.left:
                    new_relation = IndependenceRelation(relation1.left, relation1.right, relation1.given | relation2.given)
                    new_relations.add(new_relation)
        return new_relations


    def apply_contraction(self, relations: Set[IndependenceRelation]) -> Set[IndependenceRelation]:
        new_relations = set(relations)
        for relation in relations:
            if relation.left == relation.right:
                new_relation = IndependenceRelation(relation.left, relation.right, relation.given)
                new_relations.add(new_relation)
        return new_relations


# Example usage
# relations = {
#     IndependenceRelation({"A"}, {"B"}, {"C"}), # (A ⊥ B) | C
#     IndependenceRelation({"D"}, {"E"}, {"F"}), # (D ⊥ E) | F
#     IndependenceRelation({"G"}, {"H"}, {"I"})  # (G ⊥ H) | I
# }

relations2 = { 

    IndependenceRelation({"X1"}, {"X2"}, {}),
    IndependenceRelation({"X2"}, {"X1", "X4"}, {}),
    IndependenceRelation({"X3"}, {"X4"}, {"X1", "X2"}),
    IndependenceRelation({"X4"}, {"X2", "X3"}, {"X1"}),
    IndependenceRelation({"X5"}, {"X1", "X2"}, {"X3", "X4"}),
    IndependenceRelation({"X4", "X5"}, {"X2"}, {"X1", "X3"})
}


i=0
model = SemigraphoidModel(relations2)
for relations2 in model.minimized_relations:
    i+=1
    print(f"({i}):", relations2)

In [2]:
from typing import Set
from functools import cached_property

class IndependenceRelation:
    def __init__(self, left: Set, right: Set, given: Set):
        self.left = left
        self.right = right
        self.given = given

    def __eq__(self, other):
        return (self.left, self.right, self.given) == (other.left, other.right, other.given)

    def __hash__(self):
        return hash((tuple(self.left), tuple(self.right), tuple(self.given)))

    def __repr__(self):
        left_str = ", ".join(self.left)
        right_str = ", ".join(self.right)
        given_str = ", ".join(self.given)
        return f"({left_str} ⊥ {right_str}) | {given_str}"

class SemigraphoidModel:
    def __init__(self, relations: Set[IndependenceRelation]):
        self.relations = relations

    @cached_property
    def minimized_relations(self) -> Set[IndependenceRelation]:
        # Start with an empty set for the result
        minimal_relations = set()

        # Iterate over each relation in the input set
        for relation in self.relations:
            # Check if adding the current relation would create redundancy
            if not self.is_redundant(minimal_relations, relation):
                # If not redundant, add it to the minimal set
                minimal_relations.add(relation)

        return minimal_relations

    def is_redundant(self, minimal_relations, relation):
        # Check if the relation is redundant with respect to the current minimal set
        for existing_relation in minimal_relations:
            if (
                existing_relation.left.issubset(relation.left) and
                existing_relation.right.issubset(relation.right) and
                existing_relation.given.issubset(relation.given)
            ):
                return True
        return False

# Example usage
# relations2 = { 
#     IndependenceRelation({"X1"}, {"X2"}, {}),
#     IndependenceRelation({"X2"}, {"X1", "X4"}, {}),
#     IndependenceRelation({"X3"}, {"X4"}, {"X1", "X2"}),
#     IndependenceRelation({"X4"}, {"X2", "X3"}, {"X1"}),
#     IndependenceRelation({"X5"}, {"X1", "X2"}, {"X3", "X4"}),
#     IndependenceRelation({"X4", "X5"}, {"X2"}, {"X1", "X3"})
# }
    
relations2 = {
    IndependenceRelation({"X1"}, {"X2"}, {}),
    IndependenceRelation({"X2"}, {"X1"}, {}),

    IndependenceRelation({"X2"}, {"X4"}, {}),
    IndependenceRelation({"X4"}, {"X2"}, {}),

    IndependenceRelation({"X3"}, {"X4"}, {"X1", "X2"}),
    IndependenceRelation({"X3"}, {"X4"}, {"X2", "X1"}),

    IndependenceRelation({"X4"}, {"X2"}, {"X1"}),
    IndependenceRelation({"X4"}, {"X3"}, {"X1"}),

    IndependenceRelation({"X5"}, {"X1"}, {"X3", "X4"}),
    IndependenceRelation({"X2"}, {"X5"}, {"X4", "X3"}),

    IndependenceRelation({"X4"}, {"X2"}, {"X1", "X3"}),
    IndependenceRelation({"X5"}, {"X2"}, {"X1", "X3"}),
}

i = 0
model = SemigraphoidModel(relations2)
for relation in model.minimized_relations:
    i += 1
    print(f"({i}):", relation)


(1): (X2 ⊥ X4) | 
(2): (X4 ⊥ X2) | X1
(3): (X5 ⊥ X1) | X3, X4
(4): (X4 ⊥ X2) | 
(5): (X2 ⊥ X1) | 
(6): (X4 ⊥ X3) | X1
(7): (X2 ⊥ X5) | X3, X4
(8): (X1 ⊥ X2) | 
(9): (X3 ⊥ X4) | X1, X2
(10): (X5 ⊥ X2) | X1, X3


In [ ]:
import itertools
import networkx as nx
ex_dag = nx.DiGraph()
ex_dag.add_edges_from([
        ("X2", "X3"),
        ("X1", "X3"),
        ("X1", "X4"),
        ("X4", "X5"),
        ("X3", "X5"),])

In [ ]:
lst = [f"X{i}" for i in range(1, 6)]
for i in itertools.combinations(lst, 2):
    print(f"For {i[0]} -> {i[1]}")
    for path in nx.all_simple_paths(ex_dag, i[0], i[1]):
        print(path)
    print()

In [ ]:
# for i in nx.all_simple_paths(ex_dag, "X5", "X1"):
#     print(i)

# known d-separations
print(
nx.d_separated(ex_dag, {"X1"}, {"X2"}, {}),
nx.d_separated(ex_dag, {"X2"}, {"X1", "X4"}, {}),
nx.d_separated(ex_dag, {"X3"}, {"X4"}, {"X1", "X2"}),
nx.d_separated(ex_dag, {"X4"}, {"X2", "X3"}, {"X1"}),
nx.d_separated(ex_dag, {"X5"}, {"X1", "X2"}, {"X3", "X4"}),
nx.d_separated(ex_dag, {"X4", "X5"}, {"X2"}, {"X1", "X3"}),
)

In [ ]:
# nx.descendants(ex_dag, "X2")

# nx.is_directed_acyclic_graph(ex_dag)
# nx.dag_to_branching(ex_dag).nodes(data="source")

for path in nx.all_simple_paths(ex_dag, "X1", "X5"):
    print(path)